In [ ]:
import numpy as np
import os
import time

In [ ]:
!pip install tensorflow==2

     |████████████████████████████████| 86.3MB 51kB/s 
     |████████████████████████████████| 3.8MB 52.8MB/s 
     |████████████████████████████████| 450kB 52.7MB/s 
     |████████████████████████████████| 81kB 14.6MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstall

In [ ]:
from google.colab import files

In [ ]:
uploaded=files.upload()

Saving poetry.txt to poetry.txt


In [ ]:
path_to_file = "poetry.txt"

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 61794 characters


In [ ]:

print(text[:230])



اول دفتر به نام ایزد دانا
صانع پروردگار حی توانا
اکبر و اعظم خدای عالم و آدم
صورت خوب آفرید و سیرت زیبا
از در بخشندگی و بنده نوازی
مرغ هوا را نصیب و ماهی دریا
قسمت خود می‌خورند منعم و درویش
روزی خود می‌برند پشه و عنقا
حاجت موری 


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

46 unique characters


In [ ]:

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  ':' :   3,
  '«' :   4,
  '»' :   5,
  '،' :   6,
  '؟' :   7,
  'ء' :   8,
  'آ' :   9,
  'أ' :  10,
  'ؤ' :  11,
  'ئ' :  12,
  'ا' :  13,
  'ب' :  14,
  'ت' :  15,
  'ث' :  16,
  'ج' :  17,
  'ح' :  18,
  'خ' :  19,
  ...
}


In [ ]:

print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\n\nاول دفتر به' ---- characters mapped to int ---- > [ 0  0 13 38 34  1 20 32 15 22  1 14 37]


In [ ]:
# The maximum length sentence we want for a single input in characters
import tensorflow as tf
seq_length = 100

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])





ا
و
ل


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))
    print("***"*5)

'\n\nاول دفتر به نام ایزد دانا\nصانع پروردگار حی توانا\nاکبر و اعظم خدای عالم و آدم\nصورت خوب آفرید و سیرت '
***************
'زیبا\nاز در بخشندگی و بنده نوازی\nمرغ هوا را نصیب و ماهی دریا\nقسمت خود می\u200cخورند منعم و درویش\nروزی خود م'
***************
'ی\u200cبرند پشه و عنقا\nحاجت موری به علم غیب بداند\nدر بن چاهی به زیر صخره صما\nجانور از نطفه می\u200cکند شکر از ن'
***************
'ی\nبرگ\u200cتر از چوب خشک و چشمه ز خارا\nشربت نوش آفرید از مگس نحل\nنخل تناور کند ز دانه خرما\nاز همگان بی\u200cنیا'
***************
'ز و بر همه مشفق\nاز همه عالم نهان و بر همه پیدا\nپرتو نور سرادقات جلالش\nاز عظمت ماورای فکرت دانا\nخود نه'
***************


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\n\nاول دفتر به نام ایزد دانا\nصانع پروردگار حی توانا\nاکبر و اعظم خدای عالم و آدم\nصورت خوب آفرید و سیرت'
Target data: '\nاول دفتر به نام ایزد دانا\nصانع پروردگار حی توانا\nاکبر و اعظم خدای عالم و آدم\nصورت خوب آفرید و سیرت '


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 0 ('\n')
  expected output: 0 ('\n')
Step    1
  input: 0 ('\n')
  expected output: 13 ('ا')
Step    2
  input: 13 ('ا')
  expected output: 38 ('و')
Step    3
  input: 38 ('و')
  expected output: 34 ('ل')
Step    4
  input: 34 ('ل')
  expected output: 1 (' ')


In [ ]:
# Batch size
BATCH_SIZE = 64


dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 25

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model.predict(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 46) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 25)            1150      
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3228672   
_________________________________________________________________
dense (Dense)                (64, None, 46)            47150     
Total params: 3,276,972
Trainable params: 3,276,972
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

array([31,  5, 35, 25, 38, 30, 27, 21, 25, 37, 28, 20, 12, 19, 40, 25,  3,
       31, 38, 36, 43,  9,  0, 40, 33, 34, 23, 28, 36, 11, 25,  8, 37,  2,
       27, 18, 24, 45, 35,  6, 27, 20, 27, 43, 44, 24, 12, 43,  5, 21,  1,
       11, 20, 36,  2, 13,  2, 38, 16, 41, 14,  5,  2,  7, 38, 25,  0, 24,
       43, 12, 20, 33, 17,  9, 13, 15, 24,  4, 29, 34, 24, 31,  0, 21,  4,
        8, 27,  8, 45, 28, 40, 34, 24, 45, 23, 41,  4,  8, 20, 20])

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '\n\nاول دفتر به نام ایزد دانا\nصانع پروردگار حی توانا\nاکبر و اعظم خدای عالم و آدم\nصورت خوب آفرید و سیرت'

Next Char Predictions: 
 'غ»مشوعضذشهطدئخپش:غونگآ\nپقلزطنؤشءه!ضحس\u200cم،ضدضگیسئگ»ذ ؤدن!ا!وثچب»!؟وش\nسگئدقجآاتس«ظلسغ\nذ«ءضء\u200cطپلس\u200cزچ«ءدد'


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=80
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/80
9/9 [==============================] - 36s 4s/step - loss: 4.0764
Epoch 2/80
9/9 [==============================] - 35s 4s/step - loss: 3.7456
Epoch 3/80
9/9 [==============================] - 34s 4s/step - loss: 3.6446
Epoch 4/80
9/9 [==============================] - 34s 4s/step - loss: 3.1177
Epoch 5/80
9/9 [==============================] - 34s 4s/step - loss: 2.9982
Epoch 6/80
9/9 [==============================] - 34s 4s/step - loss: 2.9656
Epoch 7/80
9/9 [==============================] - 33s 4s/step - loss: 2.9440
Epoch 8/80
9/9 [==============================] - 33s 4s/step - loss: 2.9217
Epoch 9/80
9/9 [==============================] - 35s 4s/step - loss: 2.8888
Epoch 10/80
9/9 [==============================] - 34s 4s/step - loss: 2.8439
Epoch 11/80
9/9 [==============================] - 34s 4s/step - loss: 2.7889
Epoch 12/80
9/9 [==============================] - 33s 4s/step - loss: 2.7278
Epoch 13/80
9/9 [==============================] - 33s 4s/step - loss: 2.

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


In [ ]:
model.summary()

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []


  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions 
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
rint(generate_text(model, start_string=u"اول دفتر"))